# Traveling Salesman Problem

We are given a [complete graph](https://en.wikipedia.org/wiki/Complete_graph) which means that every pair of distinct vertices is connected by a unique edge. Each edge has traveling cost associated with it. We need to find a shortest route that goes throught all the nodes and comes back to the original node.

## Representation

We can represent traveling costs as a simple matrix.

In [1]:
i = 'i'
m =  [
    [i, 3, 1, 8],
    [3, i, 4, 4],
    [1, 4, i, 7],
    [8, 4, 7, i],
]

This represents 4 nodes (0 through 3) where cost of traveling from 0 to 3 is

In [2]:
m[0][3]

8

## Brute force solution

In [3]:
from itertools import permutations

def total_cost(matrix, route):
    # we are going from node a to node b in each step
    a = list(route)
    b = a[1:] + [a[0]]

    # calculate the route cost
    cost = 0
    for ii, jj in zip(a, b):
        cost += matrix[ii][jj]
    return cost


def brute_force(matrix):
    current_min = pow(10, 10) # should be at least bigger than min route
    current_route = []
    for route in permutations(range(len(matrix))):
        route_cost = total_cost(matrix, route)
        # check if we got new min route
        if route_cost < current_min:
            current_min = route_cost
            current_route = route
    
    return current_route, current_min

In [4]:
route, distance = brute_force(m)

In [5]:
print('Found route:', route, 'with distance:', distance)

Found route: (0, 1, 3, 2) with distance: 15


## Greedy solution
What if we just always pick the shortest distance from all available routes in a current node. We start with first node and pick the cheapest node that we haven't picked yet.

In [6]:
def get_min_node(nodes, visited_set):
    min_cost = pow(10, 10)
    node_position = None
    for position, cost in enumerate(nodes):
        # make sure we haven't visited the node yet, we are not trying to go to
        # the same node, and current cost is the minimum that we've seen so far
        if position in visited_set or isinstance(cost, str) or cost >= min_cost:
            continue

        min_cost = cost
        node_position = position

    return node_position

def greedy(matrix, route):
    print(route)
    current_node = route[-1]
    min_node = get_min_node(matrix[current_node], set(route))

    if min_node is None:
        return route

    return greedy(matrix, route + [min_node])
    

In [7]:
answer = greedy(m, [0])

[0]
[0, 2]
[0, 2, 1]
[0, 2, 1, 3]


In [8]:
answer

[0, 2, 1, 3]

In [9]:
total_cost(m, answer)

17

This is not optimal route but its close to brute force answer. The benifit of the algorithm is relative speed O(n<sup>2</sup>) and simplicity.

In [10]:
total_cost(m, [0, 1, 3, 2]) # route found by brute force

15

In [70]:
i = pow(10, 10)
n =  [[i,1,2,2,1], # 1
      [1,i,3,4,3], # 2
      [2,3,i,1,3], # 3
      [2,4,1,i,1], # 4
      [1,3,3,1,i] #5
]

In [71]:
g = greedy(n, [0])

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]


In [72]:
g

[0, 1, 2, 3, 4]

In [73]:
total_cost(n, g)

7

In [74]:
brute_force(n)

((0, 1, 2, 3, 4), 7)

## Branch and Bound

In [80]:
def print_matrix(matrix):
    for ii in matrix:
        new_row = []
        for jj in ii:
            if jj > 1000:
                new_row.append('i')
                continue
            new_row.append(jj)
        print(new_row)

In [84]:
def reduse(matrix, a, b):
    new_matrix = []
    min_ii = min_jj = pow(10, 10)

    for n, ii in enumerate(matrix):
        new_row = []

        for m, jj in enumerate(ii):
            if b == n or m == b:
                continue
            new_row.append(jj)

        if new_row:
            new_matrix.append(new_row)
        

    return(new_matrix)
            

In [85]:
redused = reduse(n, 1, 0)

In [86]:
print_matrix(redused)

['i', 3, 4, 3]
[3, 'i', 1, 3]
[4, 1, 'i', 1]
[3, 3, 1, 'i']


In [89]:
def min_in_row(matrix):
    vector = []
    for ii in matrix:
        min_element = pow(10, 10)
        for jj in ii:
            if min_element > jj:
                min_element = jj
        vector.append(min_element)
    return vector

In [91]:
to_subtruct = min_in_row(redused)

In [92]:
to_subtruct

[3, 1, 1, 1]

In [96]:
def subtract(matrix, vector):
    new_matrix = []
    for n, ii in enumerate(matrix):
        new_row = []
        for jj in ii:
            new_row.append(jj - vector[n])
        new_matrix.append(new_row)
    return new_matrix

In [97]:
redused_and_subtracted = subtract(redused, to_subtruct)

In [99]:
print_matrix(redused_and_subtracted)

['i', 0, 1, 0]
[2, 'i', 0, 2]
[3, 0, 'i', 0]
[2, 2, 0, 'i']


In [112]:
def min_in_column(matrix):
    vector = []
    for ii in matrix:
        print(ii)

In [113]:
min_in_column(redused_and_subtracted)

[9999999997, 0, 1, 0]
[2, 9999999999, 0, 2]
[3, 0, 9999999999, 0]
[2, 2, 0, 9999999999]
